<a href="https://colab.research.google.com/github/aniketSanyal/DifferentialPrivacy/blob/main/Homework_Assignment_3_RML_Aniket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, TensorDataset, Dataset



mnist_train = datasets.MNIST("../data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("../data", train=False, download=True, transform=transforms.ToTensor())

In [ ]:
labels_to_filter = [1, 7]

train_indices = [i for i, label in enumerate(mnist_train.targets) if label in labels_to_filter]

filtered_data = mnist_train.data[train_indices]
filtered_targets = mnist_train.targets[train_indices]

filtered_targets[filtered_targets == 1] = 0
filtered_targets[filtered_targets == 7] = 1

filtered_mnist_train = TensorDataset(filtered_data, filtered_targets)

test_indices = [i for i, label in enumerate(mnist_test.targets) if label in labels_to_filter]

filtered_data_test = mnist_test.data[test_indices]
filtered_targets_test = mnist_test.targets[test_indices]

filtered_targets_test[filtered_targets_test == 1] = 0
filtered_targets_test[filtered_targets_test == 7] = 1


filtered_mnist_test= TensorDataset(filtered_data_test, filtered_targets_test)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import random
def craft_random_lflip(train_set, ratio):
  list_train_set = list(train_set)
  num_flipped_samples = int(len(list_train_set) * ratio)
  indices_to_flip = random.sample(range(len(list_train_set)), num_flipped_samples)

  copy_targets = filtered_targets.clone().detach()

  for idx in indices_to_flip:
    copy_targets[idx] = 1- copy_targets[idx]

  flipped_train_set = TensorDataset(filtered_data, copy_targets)

  return flipped_train_set

In [ ]:
five_percent_poisoned = craft_random_lflip(filtered_mnist_train, 0.05)
ten_percent_poisoned = craft_random_lflip(filtered_mnist_train, 0.1)
twenty_five_percent_poisoned = craft_random_lflip(filtered_mnist_train, 0.25)
fifty_percent_poisoned = craft_random_lflip(filtered_mnist_train, 0.5)

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

Unpoisoned Dataset

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=filtered_mnist_train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=filtered_mnist_test, batch_size=100, shuffle=False)

In [ ]:
len(list(train_loader.dataset))

13007

In [ ]:
batch_size = 100
n_iters = 3000
epochs = n_iters / (len(list(train_loader.dataset)) / batch_size)
input_dim = 784
output_dim = 10
lr_rate = 0.001

In [ ]:
from torch.autograd import Variable

In [ ]:
model = LogisticRegression(input_dim, output_dim)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=five_percent_poisoned, batch_size=100, shuffle=True)

In [ ]:
Loss = []
acc = []
for epoch in range(int(epochs)):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images.view(-1, 28*28).float())
        loss = criterion(outputs, labels)
        # Loss.append(loss.item())
        loss.backward()
        optimizer.step()
    Loss.append(loss.item())
    correct = 0
    for images, labels in test_loader:
        outputs = model(images.view(-1, 28*28).float())
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum()
    accuracy = 100 * (correct.item()) / len(list(test_loader.dataset))
    acc.append(accuracy)
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

Epoch: 0. Loss: 0.0. Accuracy: 98.1044845122515
Epoch: 1. Loss: 4.773677349090576. Accuracy: 64.58622283865002
Epoch: 2. Loss: 47.11353302001953. Accuracy: 47.57281553398058
Epoch: 3. Loss: 0.0. Accuracy: 99.21405455386038
Epoch: 4. Loss: 0.0. Accuracy: 97.91955617198336
Epoch: 5. Loss: 318.4360656738281. Accuracy: 47.52658344891355
Epoch: 6. Loss: 107.76445770263672. Accuracy: 75.91308368007397
Epoch: 7. Loss: 47.64955520629883. Accuracy: 52.56588072122053
Epoch: 8. Loss: 0.0. Accuracy: 97.17984281091077
Epoch: 9. Loss: 83.77359771728516. Accuracy: 47.61904761904762
Epoch: 10. Loss: 0.0. Accuracy: 98.6130374479889
Epoch: 11. Loss: 0.0. Accuracy: 99.02912621359224
Epoch: 12. Loss: 48.67447280883789. Accuracy: 67.40638002773925
Epoch: 13. Loss: 0.0. Accuracy: 97.41100323624596
Epoch: 14. Loss: 0.0. Accuracy: 98.70550161812298
Epoch: 15. Loss: 0.0. Accuracy: 98.33564493758668
Epoch: 16. Loss: 0.0. Accuracy: 98.79796578825705
Epoch: 17. Loss: 0.0. Accuracy: 98.47434119278779
Epoch: 18. Lo

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=ten_percent_poisoned, batch_size=100, shuffle=True)
Loss = []
acc = []
for epoch in range(int(epochs)):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images.view(-1, 28*28).float())
        loss = criterion(outputs, labels)
        # Loss.append(loss.item())
        loss.backward()
        optimizer.step()
    Loss.append(loss.item())
    correct = 0
    for images, labels in test_loader:
        outputs = model(images.view(-1, 28*28).float())
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum()
    accuracy = 100 * (correct.item()) / len(list(test_loader.dataset))
    acc.append(accuracy)
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

Epoch: 0. Loss: 81.69593048095703. Accuracy: 50.90152565880721
Epoch: 1. Loss: 65.41656494140625. Accuracy: 98.84419787332409
Epoch: 2. Loss: 72.91947174072266. Accuracy: 60.33287101248266
Epoch: 3. Loss: 0.0. Accuracy: 95.97780859916782
Epoch: 4. Loss: 134.9026641845703. Accuracy: 67.68377253814147
Epoch: 5. Loss: 169.46560668945312. Accuracy: 89.45908460471567
Epoch: 6. Loss: 99.42271423339844. Accuracy: 48.451225150254274
Epoch: 7. Loss: 0.0. Accuracy: 96.02404068423486
Epoch: 8. Loss: 62.39168167114258. Accuracy: 50.39297272306981
Epoch: 9. Loss: 112.41795349121094. Accuracy: 53.62921867776237
Epoch: 10. Loss: 9.38681697845459. Accuracy: 90.3837263060564
Epoch: 11. Loss: 0.009561248123645782. Accuracy: 79.33425797503467
Epoch: 12. Loss: 11.082550048828125. Accuracy: 85.57558945908461
Epoch: 13. Loss: 88.77049255371094. Accuracy: 95.51548774849746
Epoch: 14. Loss: 67.0282974243164. Accuracy: 53.490522422561256
Epoch: 15. Loss: 0.0. Accuracy: 98.84419787332409
Epoch: 16. Loss: 66.099

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=twenty_five_percent_poisoned, batch_size=100, shuffle=True)
Loss = []
acc = []
for epoch in range(int(epochs)):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images.view(-1, 28*28).float())
        loss = criterion(outputs, labels)
        # Loss.append(loss.item())
        loss.backward()
        optimizer.step()
    Loss.append(loss.item())
    correct = 0
    for images, labels in test_loader:
        outputs = model(images.view(-1, 28*28).float())
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum()
    accuracy = 100 * (correct.item()) / len(list(test_loader.dataset))
    acc.append(accuracy)
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

Epoch: 0. Loss: 83.30655670166016. Accuracy: 71.15117891816921
Epoch: 1. Loss: 802.326171875. Accuracy: 49.283402681460935
Epoch: 2. Loss: 163.14610290527344. Accuracy: 97.457235321313
Epoch: 3. Loss: 71.98863220214844. Accuracy: 32.7785483125289
Epoch: 4. Loss: 232.6356201171875. Accuracy: 89.82894128525197
Epoch: 5. Loss: 154.52195739746094. Accuracy: 52.47341655108645
Epoch: 6. Loss: 152.13063049316406. Accuracy: 47.52658344891355
Epoch: 7. Loss: 625.3942260742188. Accuracy: 47.61904761904762
Epoch: 8. Loss: 479.7392883300781. Accuracy: 53.536754507628295
Epoch: 9. Loss: 708.7190551757812. Accuracy: 52.47341655108645
Epoch: 10. Loss: 1190.6201171875. Accuracy: 52.47341655108645
Epoch: 11. Loss: 6.46636962890625. Accuracy: 66.43550624133148
Epoch: 12. Loss: 44.841217041015625. Accuracy: 48.03513638465095
Epoch: 13. Loss: 148.40916442871094. Accuracy: 49.00601017105871
Epoch: 14. Loss: 32.52550506591797. Accuracy: 51.91863153028201
Epoch: 15. Loss: 409.0777893066406. Accuracy: 91.4932

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=fifty_percent_poisoned, batch_size=100, shuffle=True)
Loss = []
acc = []
for epoch in range(int(epochs)):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images.view(-1, 28*28).float())
        loss = criterion(outputs, labels)
        # Loss.append(loss.item())
        loss.backward()
        optimizer.step()
    Loss.append(loss.item())
    correct = 0
    for images, labels in test_loader:
        outputs = model(images.view(-1, 28*28).float())
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum()
    accuracy = 100 * (correct.item()) / len(list(test_loader.dataset))
    acc.append(accuracy)
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

Epoch: 0. Loss: 357.7503356933594. Accuracy: 42.57975034674064
Epoch: 1. Loss: 271.1650695800781. Accuracy: 52.24225612575127
Epoch: 2. Loss: 546.5447998046875. Accuracy: 52.47341655108645
Epoch: 3. Loss: 406.105224609375. Accuracy: 52.47341655108645
Epoch: 4. Loss: 289.3488464355469. Accuracy: 47.57281553398058
Epoch: 5. Loss: 1121.8245849609375. Accuracy: 52.47341655108645
Epoch: 6. Loss: 570.5703735351562. Accuracy: 52.47341655108645
Epoch: 7. Loss: 817.0780029296875. Accuracy: 47.52658344891355
Epoch: 8. Loss: 346.09954833984375. Accuracy: 52.47341655108645
Epoch: 9. Loss: 682.3914184570312. Accuracy: 47.942672214516875
Epoch: 10. Loss: 477.4519348144531. Accuracy: 53.536754507628295
Epoch: 11. Loss: 320.99560546875. Accuracy: 44.29033749422099
Epoch: 12. Loss: 21.632375717163086. Accuracy: 21.7753120665742
Epoch: 13. Loss: 123.30308532714844. Accuracy: 53.44429033749422
Epoch: 14. Loss: 69.49102020263672. Accuracy: 49.09847434119279
Epoch: 15. Loss: 232.7049102783203. Accuracy: 50

Task 2

In [ ]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    # Train the model for the specified number of epochs
    for epoch in range(num_epochs):
        # Set the model to train mode
        model.to(device)
        model.train()

        # Initialize the running loss and accuracy
        running_loss = 0.0
        running_corrects = 0

        # Iterate over the batches of the train loader
        for inputs, labels in train_loader:
            # Move the inputs and labels to the device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the optimizer gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Backward pass and optimizer step
            loss.backward()
            optimizer.step()

            # Update the running loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Calculate the train loss and accuracy
        train_loss = running_loss / len(list(train_loader.dataset))
        train_acc = running_corrects.double() / len(list(train_loader.dataset))

        # Set the model to evaluation mode
        model.eval()

        # Initialize the running loss and accuracy
        running_loss = 0.0
        running_corrects = 0
        running_incorrects = 0

        # Iterate over the batches of the validation loader
        with torch.no_grad():
            for inputs, labels in val_loader:
                # Move the inputs and labels to the device
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Update the running loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_incorrects += torch.sum(preds != labels.data)

        # Calculate the validation loss and accuracy
        val_loss = running_loss / len(list(val_loader.dataset))
        val_acc = running_corrects.double() / len(list(val_loader.dataset))

        # Print the epoch results
        print('Epoch [{}/{}], train loss: {:.4f}, train acc: {:.4f}, val loss: {:.4f}, val acc: {:.4f}'
              .format(epoch+1, num_epochs, train_loss, train_acc, val_loss, val_acc))

    return running_corrects, running_incorrects

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# Transform to normalize the data and convert to PyTorch tensors
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)

# Create a DataLoader for easier batch processing
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# # Extract only 'deer' and 'cat' images and labels from the train set
# deer_cat_train_indices = [i for i, label in enumerate(train_dataset.targets) if label in [3, 4]]  # 3: cat, 4: deer
# deer_cat_train_dataset = torch.utils.data.Subset(train_dataset, deer_cat_train_indices)

# # Extract only 'deer' and 'cat' images and labels from the test set
# deer_cat_test_indices = [i for i, label in enumerate(test_dataset.targets) if label in [3, 4]]  # 3: cat, 4: deer
# deer_cat_test_dataset = torch.utils.data.Subset(test_dataset, deer_cat_test_indices)

# # Create DataLoader for the subsets
# deer_cat_train_loader = torch.utils.data.DataLoader(deer_cat_train_dataset, batch_size=32, shuffle=True)
# deer_cat_test_loader = torch.utils.data.DataLoader(deer_cat_test_dataset, batch_size=32, shuffle=False)

# Print number of images in the subsets
print(f"Number of total images in the train set: {len(train_dataset)}")
print(f"Number of total images in the test set: {len(test_dataset)}")


100%|██████████| 170498071/170498071 [00:05<00:00, 29260164.88it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Number of total images in the train set: 50000
Number of total images in the test set: 10000


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

In [ ]:
deer_test_indices = [i for i, label in enumerate(test_dataset.targets) if label ==4]
deer_test_dataset = torch.utils.data.Subset(test_dataset, deer_test_indices)
deer_test_loader = torch.utils.data.DataLoader(deer_test_dataset, batch_size=32, shuffle=False)


In [45]:
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = torch.nn.Linear(resnet18.fc.in_features, 10)
# Set requires_grad to False for all parameters initially
for param in resnet18.parameters():
    param.requires_grad = True

# Set requires_grad to True for all layers except the last layer
for param in resnet18.fc.parameters():  # Last layer in the ResNet-18 architecture
    param.requires_grad = False

resnet18.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [46]:
criterion_logit = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)
train(resnet18, train_loader, test_loader, criterion_logit, optimizer, 10)

Epoch [1/10], train loss: 0.9972, train acc: 0.6583, val loss: 0.6861, val acc: 0.7678
Epoch [2/10], train loss: 0.6288, train acc: 0.7855, val loss: 0.5748, val acc: 0.8035
Epoch [3/10], train loss: 0.4870, train acc: 0.8311, val loss: 0.5461, val acc: 0.8137
Epoch [4/10], train loss: 0.3929, train acc: 0.8646, val loss: 0.5380, val acc: 0.8189
Epoch [5/10], train loss: 0.3157, train acc: 0.8914, val loss: 0.5519, val acc: 0.8238
Epoch [6/10], train loss: 0.2589, train acc: 0.9100, val loss: 0.5602, val acc: 0.8233
Epoch [7/10], train loss: 0.2146, train acc: 0.9260, val loss: 0.5814, val acc: 0.8220
Epoch [8/10], train loss: 0.1737, train acc: 0.9407, val loss: 0.5860, val acc: 0.8259
Epoch [9/10], train loss: 0.1470, train acc: 0.9494, val loss: 0.6232, val acc: 0.8187
Epoch [10/10], train loss: 0.1227, train acc: 0.9590, val loss: 0.6259, val acc: 0.8263


(tensor(8263, device='cuda:0'), tensor(1737, device='cuda:0'))

In [47]:
criterion = torch.nn.MSELoss()

In [48]:
def craft_clabel_poisons(model, target, bases, n_iter, lr, beta):
  model.eval()
  f_t = model(target.unsqueeze(0))
  poisons = []
  for base in bases:
    x = base.clone().detach()
    x.requires_grad = True
    for i in range(n_iter):
      x.requires_grad = True

      loss = criterion(model(x.unsqueeze(0)) , f_t)
      loss.backward(retain_graph = True)
      print(x.grad)
      x_i = x.clone().detach() - lr * x.grad
      x = (x_i + lr*beta* base)/(1+beta*lr)
    poisons.append(x)
  return poisons

In [49]:
import random

In [50]:
deer_indices = [index for index, label in enumerate(test_dataset.targets) if label == 4]  # 4 corresponds to deer
cat_indices = [index for index, label in enumerate(test_dataset.targets) if label == 3]   # 3 corresponds to cat

target_deer_indices = random.sample(deer_indices, 5)
base_cat_indices = random.sample(cat_indices, 100)

In [51]:
target_deer_dataset = torch.utils.data.Subset(test_dataset, target_deer_indices)
base_cat_dataset = torch.utils.data.Subset(test_dataset, base_cat_indices)

In [52]:
target_deer_tensors = []
for (tensor, label) in list(target_deer_dataset):
  target_deer_tensors.append(tensor)

base_cat_tensors = []
for (tensor, label) in list(base_cat_dataset):
  base_cat_tensors.append(tensor)


In [53]:
class PoisonedCIFAR10(Dataset):
    def __init__(self, original_dataset, additional_data):
        self.original_dataset = original_dataset
        self.additional_data = additional_data

        # Calculate the total length of the extended dataset
        self.total_length = len(original_dataset) + len(additional_data)

    def __len__(self):
        return self.total_length

    def __getitem__(self, index):
        if index < len(self.original_dataset):
            return self.original_dataset[index]
        else:
            adjusted_index = index - len(self.original_dataset)
            tensor, label = self.additional_data[adjusted_index]
            return tensor, label



In [54]:
list_of_poisons = []
list_of_datasets = []
number_of_poisons = [1,5,10,25,50,100]
resnet18.to("cpu")
for target in target_deer_tensors:
  poisons = craft_clabel_poisons(resnet18, target, base_cat_tensors, 10, 0.1, 0.01)
  poisons_with_labels = []
  for poison in poisons:
   poisons_with_labels.append((poison,3))
  for k in number_of_poisons:
    selected_poisons  = random.sample(poisons_with_labels, k)
    poisoned_dataset = PoisonedCIFAR10(train_dataset, selected_poisons)
    list_of_datasets.append(poisoned_dataset)

Streaming output truncated to the last 5000 lines.
         [ 0.0207,  0.0220,  0.0741,  ...,  0.2739, -0.0603,  0.0928],
         [-0.1537,  0.0310, -0.0055,  ...,  0.1189, -0.0113,  0.1322]]])
tensor([[[ 0.0662, -0.0404, -0.2451,  ...,  0.0509, -0.0878,  0.0122],
         [-0.0534,  0.0453, -0.1847,  ...,  0.1779, -0.0126,  0.0162],
         [ 0.0769,  0.1509, -0.0047,  ..., -0.0330, -0.0429, -0.0103],
         ...,
         [-0.2284, -0.3963,  0.4210,  ..., -0.1693, -0.1047, -0.0144],
         [-0.0499, -0.3281, -0.0493,  ..., -0.1131, -0.0382, -0.0451],
         [-0.1268, -0.1864, -0.1977,  ..., -0.0401,  0.0162,  0.0111]],

        [[ 0.1241,  0.0157, -0.3131,  ...,  0.0914, -0.0420,  0.0570],
         [-0.0639,  0.1329, -0.2070,  ..., -0.0079,  0.0471,  0.0698],
         [ 0.0253,  0.1898, -0.0435,  ...,  0.0460,  0.0300,  0.0352],
         ...,
         [ 0.0492, -0.1762,  0.4434,  ..., -0.1264, -0.0373, -0.0043],
         [ 0.0754, -0.2572, -0.0197,  ..., -0.0439, -0.0349, -0.0

In [55]:
for param in resnet18.parameters():
    param.requires_grad = False


for param in resnet18.fc.parameters():
    param.requires_grad = True

In [56]:
resnet18.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [57]:
for i, poisoned_dataset in enumerate(list_of_datasets):
  poisoned_dataloader = torch.utils.data.DataLoader(poisoned_dataset, batch_size=32, shuffle=True)
  print("Number of poisons added : ", number_of_poisons[i%6])
  failure_attack, success_attack = train(resnet18, poisoned_dataloader, deer_test_loader, criterion_logit, optimizer, 10)
  print("Succesful Attacks = ", success_attack.item())
  print("Percentage of successful attacks = ", (success_attack.item()/(success_attack.item() + failure_attack.item())))
  print(" -----------------------------------------------------------------------------")


Number of poisons added :  1
Epoch [1/10], train loss: 0.0890, train acc: 0.9701, val loss: 0.8923, val acc: 0.7970
Epoch [2/10], train loss: 0.0862, train acc: 0.9706, val loss: 0.6569, val acc: 0.8350
Epoch [3/10], train loss: 0.0800, train acc: 0.9724, val loss: 0.7832, val acc: 0.8240
Epoch [4/10], train loss: 0.0823, train acc: 0.9720, val loss: 0.6455, val acc: 0.8490
Epoch [5/10], train loss: 0.0812, train acc: 0.9722, val loss: 0.8661, val acc: 0.8040
Epoch [6/10], train loss: 0.0794, train acc: 0.9727, val loss: 0.8485, val acc: 0.8160
Epoch [7/10], train loss: 0.0801, train acc: 0.9718, val loss: 0.8027, val acc: 0.8150
Epoch [8/10], train loss: 0.0752, train acc: 0.9744, val loss: 0.7526, val acc: 0.8310
Epoch [9/10], train loss: 0.0793, train acc: 0.9725, val loss: 0.7147, val acc: 0.8370
Epoch [10/10], train loss: 0.0795, train acc: 0.9731, val loss: 1.0182, val acc: 0.7740
Succesful Attacks =  226
Percentage of successful attacks =  0.226
 --------------------------------